In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Load the countries_IBAN File that is on git

df = pd.read_csv('/content/drive/MyDrive/CRP/countries_IBAN.csv', header = None)
df = df.rename(columns={
    df.columns[0]: 'Country',
    df.columns[1]: 'Nb_digits',
    df.columns[2]: 'code',
    df.columns[3]: 'Format',
    df.columns[4]: 'Instructions'
})
countries = df['Country']
def get_information(country):
  dicti = {}
  filter_df = df[df['Country'] == country]
  num_char = list(filter_df['Nb_digits'])[0]
  pattern = str(list(filter_df.iloc[:, 3])[0])
  comments = str(list(filter_df.iloc[:, -1])[0])
  comments = comments.replace("(", "")
  comments = comments.replace(")", "")
  comments = comments.replace('"', '')
  if 'always' in comments.split() : 
    for i in comments.split() :
      if i.isnumeric():
        number = i 
        condition = True
  else :
    condition = False
  dicti['country'] = country
  if condition:
      pattern = pattern[:2] + number + pattern[4:]
  dicti['pattern']=pattern
  dicti['number_char']=num_char
  return dicti

In [ ]:
def generate_accounts(information):
  dicti = {}
  dicti['country'] = information['country']
  dicti['pattern'] = information['pattern']
  output = list(information['pattern'])
  for i in range(len(output)):
    if output[i].islower():
      number = np.random.randint(10)
      output[i] = str(number)
  output = ''.join(output)
  dicti['output'] = output
  return dicti

In [ ]:
def generate_IBAN():
  information = []
  countries = df['Country']
  for i in countries:
    dicti = get_information(i)
    dicti = generate_accounts(dicti)
    information.append(dicti)
  return information

In [ ]:
def generate_dataset(number):
  output = []
  countries = list(df['Country'])
  for i in range(number):
    if i%100000 == 0:
      print(i)
    n = np.random.randint(len(countries))
    country = countries[n]
    dicti = get_information(country)
    dicti = generate_accounts(dicti)
    output.append((country, dicti['output']))
  return output

In [ ]:
array = np.array(generate_dataset(1500000))

In [ ]:
df = pd.DataFrame(columns=['Country', 'Iban'])
df['Country'] = [array[i][0] for i in range(len(array))]
df['Iban'] = [array[i][1] for i in range(len(array))]

In [ ]:
# Save the dataframe to a csv file
df.to_csv('/content/drive/MyDrive/CRP/Iban.csv', index = None)